In [1]:

import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# Initialize the console

# Step 2: Import necessary libraries
from typing import Optional, Dict, Any
import os
import torch
import pytorch_lightning as pl
import pandas as pd
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from rich.traceback import install
from pytorch_lightning.loggers import WandbLogger
import torch.nn as nn
import torch.optim as optim
from transformers import AutoConfig
from adapters import AutoAdapterModel, AdapterConfig
from adapters.composition import Stack
from pytorch_lightning import seed_everything, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
import torchmetrics

install(show_locals=True)

from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload

from datasets import load_from_disk

print(config.Config.TXT_SAVE_PATH)
print(config.Config.MODEL_SAVE_PATH)

dataset = load_from_disk(f"../{config.Config.DATASETS_SAVE_PATH}/datasets")


['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mixed/government', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/tmp/tmpk_v5fhyt', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-14 14:24:32.433134: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-14 14:24:32.609219: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-14 14:24:33.341926: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


./text-files/
./hp-model-


In [3]:
# Step 4: Define the DomainTaskAdapter class



class DomainTaskAdapter(pl.LightningModule):
    def __init__(self, hparams):
        super(DomainTaskAdapter, self).__init__()
        self.save_hyperparameters(hparams)
        self.config = AutoConfig.from_pretrained(self.hparams["pretrained_model_name"])
        self.config.output_hidden_states = True
        self.model = AutoAdapterModel.from_pretrained(self.hparams["pretrained_model_name"], config=self.config)
              # Define the tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.hparams["pretrained_model_name"])
        self.n = self.hparams["source_len"]  # Number of labeled source samples
        self.m = self.hparams["target_len"]  # Number of unlabeled target samples
        self.lambda_factor = self.n / (self.n + self.m)
        
        self.reduction_factor = self.hparams.get("reduction_factor", 16)
        if self.reduction_factor == "None":
            self.reduction_factor = 16
        self.leave_out = self.hparams.get("leave_out", [])
       
        self.saved_adapter_dir = self.hparams["saved_adapter_dir"]
        self.domain_adapter_name = self.hparams["domain_adapter_name"]
        self.task_adapter_name = self.hparams["task_adapter_name"]
        
        adapter_config = AdapterConfig.load("lora", r=8, alpha=16)
        self.model.add_adapter(self.task_adapter_name, config=adapter_config)
        self.model.load_adapter(f"{self.saved_adapter_dir}/{self.domain_adapter_name}")
        self.model.add_classification_head(self.task_adapter_name, num_labels=self.hparams["num_classes"])

        #self.model.add_classification_head(self.task_adapter_name, num_labels=self.hparams["num_classes"])
        self.model.active_adapters = Stack(self.domain_adapter_name, self.task_adapter_name)

        self.model.train_adapter(Stack(self.domain_adapter_name, self.task_adapter_name))
        print(self.model.adapter_summary())
        print(fn.print_trainable_parameters(self.model))
        print(self.model.active_head)


        self.training_outputs = []
        self.validation_outputs = []
        self.test_outputs = []
        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(task='multiclass',                                           
                                     num_classes=self.hparams["num_classes"])
        self.f1 = torchmetrics.F1Score(task='multiclass',num_classes=self.hparams["num_classes"], average="macro")
        self.softmax = nn.Softmax(dim=1)
    def forward(self, input_ids, attention_mask):
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits

    def compute_mlm_loss(self, input_ids, attention_mask):
        labels = input_ids.clone()
        probability_matrix = torch.full(labels.shape, 0.15)
        special_tokens_mask = [
            self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels
        ]
        probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens
        input_ids[masked_indices] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)
        self.model.active_head = self.domain_adapter_name
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        return outputs.loss
    
    def training_step(self, batch, batch_idx):
        input_ids, attention_mask = batch["source_input_ids"], batch["source_attention_mask"]
        labels = batch["label_source"]
        self.model.active_head = self.task_adapter_name

        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        loss = self.criterion(logits, labels)
        accuracy = self.accuracy(labels, torch.argmax(self.softmax(logits), dim=1))
        f1 = self.f1(labels, torch.argmax(self.softmax(logits), dim=1))
                # Target domain data for MLM
        target_input_ids, target_attention_mask = batch["target_input_ids"], batch["target_attention_mask"]
                # Forward pass for MLM task
        mlm_loss = self.compute_mlm_loss(target_input_ids, target_attention_mask)
                # Mixed loss
        mixed_loss = self.lambda_factor * loss + (1 - self.lambda_factor) * mlm_loss

        self.log("train_classification_loss", loss)
        self.log("train_mlm_loss", mlm_loss)
        self.log("train_mixed_loss", mixed_loss)

        self.log("train_accuracy", accuracy)
        self.log("train_f1", f1)
        return mixed_loss


    def validation_step(self, batch, batch_idx):
        """validation step of DomainTaskAdapter"""
        # get the input ids and attention mask for source data
        input_ids, attention_mask = batch["source_input_ids"], batch["source_attention_mask"]
        self.model.active_head = self.task_adapter_name

        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        labels = batch["label_source"]
        source_loss = self.criterion(logits, labels)
        source_accuracy = self.accuracy(labels, torch.argmax(self.softmax(logits), dim=1))
        source_f1 = self.f1(labels, torch.argmax(self.softmax(logits), dim=1))

        # get the input ids and attention mask for target data
        input_ids, attention_mask = batch["target_input_ids"], batch["target_attention_mask"]

        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        labels = batch["label_target"]
        target_loss = self.criterion(logits, labels)
        target_accuracy = self.accuracy(labels, torch.argmax(self.softmax(logits), dim=1))
        target_f1 = self.f1(labels, torch.argmax(self.softmax(logits), dim=1))
        # Target domain data for MLM
        mlm_loss = self.compute_mlm_loss(input_ids, attention_mask)
             # Mixed validation loss
        mixed_val_loss = self.lambda_factor * source_loss + (1 - self.lambda_factor) * mlm_loss
        # this will log the mean div value across epoch
        self.log("target_val/mlm_loss", mlm_loss)
        self.log("source_val/mixed_val_loss", mixed_val_loss)
        self.log(name="source_val/loss", value=source_loss, prog_bar=True, logger=True)
        self.log(name="source_val/accuracy", value=source_accuracy, prog_bar=True, logger=True)
        self.log(name="target_val/loss", value=target_loss, prog_bar=True, logger=True)
        self.log(name="target_val/accuracy", value=target_accuracy, prog_bar=True, logger=True)
        self.log(name="target_val/f1", value=target_f1, prog_bar=True, logger=True)
        self.log(name="source_val/f1", value=source_f1, prog_bar=True, logger=True)
        self.validation_outputs.append({
            "source_val/loss": source_loss,
            "source_val/accuracy": source_accuracy,
            "source_val/f1": source_f1,
            "target_val/loss": target_loss,
            "target_val/accuracy": target_accuracy,
            "target_val/f1": target_f1,
            "target_val/mlm_loss": mlm_loss,
            "source_val/mixed_val_loss": mixed_val_loss,
            
            })
        return {
            "source_val/loss": source_loss,
            "source_val/accuracy": source_accuracy,
            "source_val/f1": source_f1,
            "target_val/loss": target_loss,
            "target_val/accuracy": target_accuracy,
            "target_val/f1": target_f1,
             "target_val/mlm_loss": mlm_loss,
            "source_val/mixed_val_loss": mixed_val_loss,
        }    
    def on_validation_epoch_start(self):
        self.validation_outputs = []
    
    def on_validation_epoch_end(self):
        outputs= self.validation_outputs
        mean_source_loss = torch.stack([x["source_val/loss"] for x in outputs]).mean()
        mean_source_accuracy = torch.stack([x["source_val/accuracy"] for x in outputs]).mean()
        mean_source_f1 = torch.stack([x["source_val/f1"] for x in outputs]).mean()

        mean_target_loss = torch.stack([x["target_val/loss"] for x in outputs]).mean()
        mean_target_accuracy = torch.stack([x["target_val/accuracy"] for x in outputs]).mean()
        mean_target_f1 = torch.stack([x["target_val/f1"] for x in outputs]).mean()
        mean_mixed_loss= torch.stack([x["source_val/mixed_val_loss"] for x in outputs]).mean()
        print(f"target_val/loss: {mean_target_loss}")
        print(f"target_val/accuracy: {mean_target_accuracy}")
        print(f"target_val/f1: {mean_target_accuracy}")
        print(f"source_val/loss: {mean_target_loss}")
        print(f"source_val/accuracy: {mean_target_accuracy}")
        print(f"source_val/f1: {mean_target_accuracy}")
        print(f"mean_mixed_loss: {mean_mixed_loss}")
        # this will log the mean div value across epoch
        self.log(name="source_val/loss", value=mean_source_loss, prog_bar=True, logger=True)
        self.log(name="source_val/accuracy", value=mean_source_accuracy, prog_bar=True, logger=True)

        self.log(name="source_val/f1", value=mean_source_f1, prog_bar=True, logger=True)
        self.log(name="source_val/mixed_val_loss", value=mean_mixed_loss, prog_bar=True, logger=True)

        self.log(name="target_val/loss", value=mean_target_loss, prog_bar=True, logger=True)
        self.log(name="target_val/accuracy", value=mean_target_accuracy, prog_bar=True, logger=True)
        self.log(name="target_val/f1", value=mean_target_f1, prog_bar=True, logger=True)
                     # Log `val_loss` as `mean_source_loss`
        self.log("val_loss", mean_mixed_loss)



    def test_step(self, batch, batch_idx):
        """validation step of DomainTaskAdapter"""
        # get the input ids and attention mask for source data
        input_ids, attention_mask = batch["source_input_ids"], batch["source_attention_mask"]
        self.model.active_head = self.task_adapter_name

        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        labels = batch["label_source"]
        source_loss = self.criterion(logits, labels)
        source_accuracy = self.accuracy(labels, torch.argmax(self.softmax(logits), dim=1))
        source_f1 = self.f1(labels, torch.argmax(self.softmax(logits), dim=1))

        # get the input ids and attention mask for target data
        input_ids, attention_mask = batch["target_input_ids"], batch["target_attention_mask"]
        logits = self(input_ids=input_ids, attention_mask=attention_mask)
        labels = batch["label_target"]
        target_loss = self.criterion(logits, labels)
        target_accuracy = self.accuracy(labels, torch.argmax(self.softmax(logits), dim=1))
        target_f1 = self.f1(labels, torch.argmax(self.softmax(logits), dim=1))

        # this will log the mean div value across epoch
        self.log(name="source_test/loss", value=source_loss)
        self.log(name="source_test/accuracy", value=source_accuracy)
        self.log(name="target_test/loss", value=target_loss)
        self.log(name="target_test/accuracy", value=target_accuracy)
        self.log(name="target_test/f1", value=target_f1)
        self.log(name="source_test/f1", value=source_f1)
        self.test_outputs.append({
            "source_test/loss": source_loss,
            "source_test/accuracy": source_accuracy,
            "source_test/f1": source_f1,
            "target_test/loss": target_loss,
            "target_test/accuracy": target_accuracy,
            "target_test/f1": target_f1,
         })
        # need not to log here (or we can do it but let's log at the end of each epoch)
        return {
            "source_test/loss": source_loss,
            "source_test/accuracy": source_accuracy,
            "source_test/f1": source_f1,
            "target_test/loss": target_loss,
            "target_test/accuracy": target_accuracy,
            "target_test/f1": target_f1,
        }
    def on_test_epoch_start(self):
        self.test_outputs = []
    def on_test_epoch_end(self):
        outputs=  self.test_outputs
        mean_source_loss = torch.stack([x["source_test/loss"] for x in outputs]).mean()
        mean_source_accuracy = torch.stack([x["source_test/accuracy"] for x in outputs]).mean()
        mean_source_f1 = torch.stack([x["source_test/f1"] for x in outputs]).mean()

        mean_target_loss = torch.stack([x["target_test/loss"] for x in outputs]).mean()
        mean_target_accuracy = torch.stack([x["target_test/accuracy"] for x in outputs]).mean()
        mean_target_f1 = torch.stack([x["target_test/f1"] for x in outputs]).mean()

        # this will log the mean div value across epoch
        self.log(name="source_test/loss", value=mean_source_loss)
        self.log(name="source_test/accuracy", value=mean_source_accuracy)
        self.log(name="target_test/loss", value=mean_target_loss)
        self.log(name="target_test/accuracy", value=mean_target_accuracy)
        self.log(name="target_test/f1", value=mean_target_f1)
        self.log(name="source_test/f1", value=mean_source_f1)
    def save_adapter(self, location, adapter_name):
        self.model.save_adapter(location, adapter_name)
    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=self.hparams["learning_rate"])
        lr_scheduler = {
            'scheduler': optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, threshold=0.0001, cooldown=0, min_lr=1e-8),
            'monitor': 'val_loss'
        }
        return [optimizer], [lr_scheduler]


In [4]:
# Step 5: Training and Evaluation Loop with Wandb logging
import wandb
wandb.login()
# Wandb setup and training loop
seeds = [42,10,100]  # List of seeds
project_name = 'mix'  # Replace with your wandb project name
domain = 'telephone_government_f1'  # Replace with the specific domain for this notebook
type = 'invLora_stack'  # Replace with the specific type for this notebook
domain_aprev ='TEG'
results = {
    "source_val/loss": [],
    "source_val/accuracy": [],
    "source_val/f1": [],
    "target_val/loss": [],
    "target_val/accuracy": [],
    "target_val/f1": [],
    "source_test/loss": [],
    "source_test/accuracy": [],
    "source_test/f1": [],
    "target_test/loss": [],
    "target_test/accuracy": [],
    "target_test/f1": [],
}

best_val_loss = float('inf')
best_model = None
best_model_path = ""


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mrawhani5 (mrawhani). Use `wandb login --relogin` to force relogin


In [5]:
reload(processed)
for seed in seeds:
    #wandb.init(project=project_name, name=f'{domain}_{type}_run_with_seed_{seed}', config={'seed': seed})

    try:
    

        seed_everything(seed)

        # Hyperparameters
        hparams = {
            "source_target": "aug_telephone_government_f1",
            "source_domain": "telephone",
            "target_domain": "government",
            "domain_adapter_name": "mlm_inv_G_aug",
            "task_adapter_name": "mixed_aug_TEG_task_invLora_f1",
            "pretrained_model_name": "bert-base-uncased",
            "padding": "max_length",
            "max_seq_length": 128,
            "bsz": 32,
            "num_classes": 3,
            "learning_rate": 1e-4,
            "reduction_factor": 16,
           
            "mode": "domain",
            "saved_adapter_dir": "../../saved/adapters",
        }
        dm = processed.DataModuleSourceTarget(hparams)
        dm.setup('fit')
        dm.setup("test")

        

        train_loader = dm.train_dataloader()
        val_loader = dm.val_dataloader()


        hparams['source_len'] = len(train_loader.dataset.source_df)
        hparams['target_len'] = len(train_loader.dataset.target_df)


        model = DomainTaskAdapter(hparams)

        checkpoint_callback = ModelCheckpoint(
            dirpath="checkpoints",
            filename="mixed_aug_TEG_task_invLora-{epoch:02d}-{val_loss:.2f}",
            save_top_k=1,
            monitor="target_val/f1",
            mode="max",
        )

        early_stopping_callback = EarlyStopping(
            monitor="target_val/f1",
            patience=3,
            mode="max",
        )
        wandb_logger = WandbLogger()
        

    except Exception as e:
        print(f"Error during preprocessing : {e}")   

    try:
      
        trainer = Trainer(
            max_epochs=10,
            accelerator="auto",
            logger=wandb_logger,
            callbacks=[checkpoint_callback, early_stopping_callback],
            limit_train_batches=1.0,
            limit_val_batches=1.0,
            limit_test_batches=1.0,
            log_every_n_steps=5,
        )
        trainer.fit(model, train_loader, val_loader)
    except Exception as e:
        print(f"Error during training : {e}")

    try:
        best_ckpt_path = checkpoint_callback.best_model_path
        model = DomainTaskAdapter.load_from_checkpoint(best_ckpt_path)
        dm.setup("test")
        test_loader = dm.test_dataloader()
        test_results = trainer.test(model, test_loader)
        print("Test Results:", test_results)

        # Collect the results for averaging later
        for key in results.keys():
            results[key].append(wandb.run.summary[key])

        dm.setup("fit")
        val_loader = dm.val_dataloader()

        # Get the metrics for the best epoch
        best_metrics = trainer.validate(model, dataloaders=val_loader, verbose=False)[0]
          # Log the best metrics to wandb summary
        for key, value in best_metrics.items():
            wandb.run.summary[key] = value

      
        # Append validation metrics to results dictionary
        for key in best_metrics.keys(): 
            if key not in results:
                results[key] = []
            results[key].append(best_metrics[key])

        # Save the best model based on validation loss
        current_val_loss = wandb.run.summary["source_val/loss"]
        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            best_model = model
            best_model_path = best_ckpt_path

    except Exception as e:
        print(f"Error during testing: {e}")

    # Finish the wandb run
    #wandb.finish()


Seed set to 42


/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


prinssst: telephone
print: government
print: 69615
prinssst: telephone
print: government
print: 69615


/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configur

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mixed_aug_TEG_task_invLora_f1lora                294,912       0.269       1       1
mlm_inv_G_aug            bottleneck        7,387,776       6.748       1       1
--------------------------------------------------------------------------------
Full model                               109,482,240     100.000               0
trainable params: 9520887 || all params: 119003127 || trainable%: 8.000535145601678
None
mixed_aug_TEG_task_invLora_f1


...

Some weights of BertAdapterModel were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['heads.default.3.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
`Trainer(limit_val_batches=1.0)` was configured so 100% of the batches will be used..
`Trainer(limit_test_batches=1.0)` was configured so 100% of the batches will be used..
/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mixed_aug_TEG_task_invLora_f1lora                294,912       0.269       1       1
mlm_inv_G_aug            bottleneck        7,387,776       6.748       1       1
--------------------------------------------------------------------------------
Full model                               109,482,240     100.000               0
trainable params: 9520887 || all params: 119003127 || trainable%: 8.000535145601678
None
mixed_aug_TEG_task_invLora_f1


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

target_val/loss: 1.1008328199386597
target_val/accuracy: 0.359375
target_val/f1: 0.359375
source_val/loss: 1.1008328199386597
source_val/accuracy: 0.359375
source_val/f1: 0.359375
mean_mixed_loss: 1.427965760231018


Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
results.items()

dict_items([('source_val/loss', [0.6322579979896545, 0.5255373120307922, 0.6442208290100098, 0.5512304306030273, 0.6343231201171875, 0.5374744534492493]), ('source_val/accuracy', [0.8053246736526489, 0.8013215661048889, 0.8026129007339478, 0.790266752243042, 0.7953029274940491, 0.795381486415863]), ('source_val/f1', [0.796276867389679, 0.7928647994995117, 0.7956877946853638, 0.7841280698776245, 0.7883199453353882, 0.7892138957977295]), ('target_val/loss', [0.6660032272338867, 0.5581586360931396, 0.650418221950531, 0.5775623321533203, 0.6419711709022522, 0.5709582567214966]), ('target_val/accuracy', [0.7856460809707642, 0.7883578538894653, 0.786083996295929, 0.778285562992096, 0.7870160341262817, 0.7833722233772278]), ('target_val/f1', [0.77858966588974, 0.781687319278717, 0.7799392938613892, 0.7721776962280273, 0.7806777358055115, 0.7769826650619507]), ('source_test/loss', [0.5530766844749451, 0.5752528309822083, 0.5713858008384705]), ('source_test/accuracy', [0.7916598320007324, 0.793

In [ ]:

# Calculate mean results
mean_results = {key: sum(values) / len(values) for key, values in results.items()}

# Log mean results to wandb
wandb.init(project=project_name, name=f'{domain}_mean_results')
for key, value in mean_results.items():
    wandb.log({key: value})
wandb.finish()

print("Mean Results:", mean_results)

# Save the best model's adapter
if model:
    adapter_save_path = "../../saved/adapter_after_run"
    model.save_adapter(adapter_save_path, hparams['task_adapter_name'])
    print(f"Adapter saved to {adapter_save_path}")
else:
    print("No best model to save.")

source_test/accuracy,▁
source_test/f1,▁
source_test/loss,▁
source_val/accuracy,▁
source_val/f1,▁
source_val/loss,▁
source_val/mixed_val_loss,▁
target_test/accuracy,▁
target_test/f1,▁
target_test/loss,▁
target_val/accuracy,▁


Mean Results: {'source_val/loss': 0.5875073571999868, 'source_val/accuracy': 0.79836838444074, 'source_val/f1': 0.7910818954308828, 'target_val/loss': 0.6108453075091044, 'target_val/accuracy': 0.784793625275294, 'target_val/f1': 0.7783423960208893, 'source_test/loss': 0.5665717720985413, 'source_test/accuracy': 0.791830579439799, 'source_test/f1': 0.7835356195767721, 'target_test/loss': 0.5855254530906677, 'target_test/accuracy': 0.7785587112108866, 'target_test/f1': 0.7732509175936381, 'target_val/mlm_loss': 1.5557419538497925, 'source_val/mixed_val_loss': 1.0489232063293457, 'val_loss': 1.0489232063293457}
Adapter saved to ../../saved/adapter_after_run


In [ ]:
print('dones')

dones


In [ ]:
best_val_loss

0.6173854470252991